# VALIDAÇÃO CRUZADA EM K-VIAS

In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

In [12]:
ds = load_iris(as_frame=True)
df = ds.frame
X = ds.data.values
y = ds.target.values

In [30]:
def selecionar_melhor_k(ks, X_treino, X_val, y_treino, y_val):
    # FAZER ASSIM TA certo FI
    acuracias_val = []
        
    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_treino, y_treino)
        pred = knn.predict(X_val)
        acuracias_val.append(accuracy_score(y_val, pred))


    melhor_acuracia_val = max(acuracias_val)
    melhor_k = ks[np.argmax(acuracias_val)]
    knn = KNeighborsClassifier(n_neighbors=melhor_k)
    knn.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
    
    return knn, melhor_k, melhor_acuracia_val

In [32]:
k_vias = 10
acuracias = []
skf = StratifiedKFold(n_splits=k_vias, shuffle=True, random_state=1)

for idx_treino, idx_teste in skf.split(X, y):
    X_treino = X[idx_treino]
    y_treino = y[idx_treino]
    
    X_teste = X[idx_teste]
    y_teste = y[idx_teste]
    
    X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.2,
                                                       stratify=y_treino, shuffle=True, random_state=1)
    
    ss = StandardScaler()
    ss.fit(X_treino)
    X_treino = ss.transform(X_treino)
    X_teste = ss.transform(X_teste)
    X_val = ss.transform(X_val)
    
    knn, _, _ = selecionar_melhor_k(range(1, 30, 2), X_treino, X_val, y_treino, y_val)
    acuracias.append(accuracy_score(y_teste, knn.predict(X_teste)))
    
print(min(acuracias))
print(max(acuracias))
print(np.mean(acuracias))
print(np.std(acuracias))

0.8666666666666667
1.0
0.9400000000000001
0.04666666666666665


# VALIDAÇÃO CRUZADA EM K-VIAS EM DOIS NIVEIS

In [35]:
from sklearn.model_selection import GridSearchCV

In [42]:
k1 = 10
k2 = 5
acuracias = []
skf = StratifiedKFold(n_splits=k1, shuffle=True, random_state=1)

for idx_treino, idx_teste in skf.split(X, y):
    X_treino = X[idx_treino]
    y_treino = y[idx_treino]
    
    X_teste = X[idx_teste]
    y_teste = y[idx_teste]
    
    ss = StandardScaler()
    ss.fit(X_treino)
    X_treino = ss.transform(X_treino)
    X_teste = ss.transform(X_teste)
    X_val = ss.transform(X_val)
    
    params = {'n_neighbors': range(1, 30, 2)}
    knn = KNeighborsClassifier()
    knn = GridSearchCV(knn, params, cv=StratifiedKFold(n_splits=k2))
    knn.fit(X_treino, y_treino)
    
    
    acuracias.append(accuracy_score(y_teste, knn.predict(X_teste)))
    
print(min(acuracias))
print(max(acuracias))
print(np.mean(acuracias))
print(np.std(acuracias))

0.8666666666666667
1.0
0.9533333333333334
0.04268749491621898
